In [ ]:
# Once we have cleaned data, we have approximately ~6.5Mil rows available.
# for the accelerated small scale models we need to strip the data down to N number of rows

# - Load Data from Processed csv filed into Dataframe
# - Param n_r : Draw n_r number of rows per N class (in this case the classes are ratings)
# - Param test_split_percentage : Specify a percentage of rows to be taken from drawn samples (eg. 5, 10, 30)
# - We can filter the decision to choose first n_r rows per class by limiting to those rows that have length under 50th or 25th percentile
# --> in essence pick smaller rows since there is ample class imbalance already

# Last recorded distribution
# 5.0-4.0    5,460,975
# 3.0        504,781
# 1.0-2.0    773,834

# DO NOT EDIT/ DELETE THIS BLOCK;


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab or local 
%cd /content/drive/My\ Drive/SNLP\ Project
# %cd Source/repos/Sentiment-Analysis-using-Deep-Learning

In [ ]:
# list content of drive - verify you are where you are supposed to be
%ls

In [ ]:
import os
import gzip
import pandas as pd
import numpy as np
import matplotlib as mpl

# Define Globals here 
ZipCSVFileName = 'abc.zip'
N_R = 2000
test_split_percentage = 10

In [ ]:
df = pd.read_csv(ZipCSVFileName)
df.info()